 This notebook reads in CVS data from [Spenvis AP-8 model](https://www.spenvis.oma.be/), and analyzes the data.  It plots the proton spectra at various points of the orbit, and performs other analysis

 Author:  Areg Danagoulian  
 Date:    04.24.2025
 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def integrate_spectrum_byEq1(bin_centers,bin_contents, threshold=200):
    """
    Compute the integral of a spectrum given its bin centers (x) and
    bin contents (y). For x-values greater than 'threshold', multiply
    the bin content by [50*(0.001*x - 0.122)] prior to integrating.
    
    Parameters
    ----------
    bin_centers : array-like
        The x-values corresponding to the center of each bin.
    bin_contents : array-like
        The y-values (counts or function values) for each bin.
    threshold : float, optional
        x-value above which y is rescaled by the factor 50*(0.001*x - 0.122).
        Default is 1.37.

    Returns
    -------
    float
        Approximate integral of the (modified) spectrum.
    """

    # Convert to numpy arrays and sort in ascending x
    x = np.asarray(bin_centers)
    y = np.asarray(bin_contents)
    
    sort_idx = np.argsort(x)
    x = x[sort_idx]
    y = y[sort_idx]

    if len(x) < 2:
        raise ValueError("Need at least two points to integrate.")

    # -------------------------
    # Step 1: Modify y-values for x > threshold
    #         factor = 50*(0.001*x - 0.122)
    #         using Eq. 1 from Carpenter, doing this in GeV
    # -------------------------
    y_mod = y.copy()
    mask = (x >= threshold)
    y_mod[~mask] = 0
    y_mod[mask] *= 50 * (0.001 * x[mask] - 0.120) #this will be neutrons/cm^2/MeV/s
    y_mod[mask] *= 50*20 #multiply by the estimated size, in cm^2, of the physics package
#    print(y_mod)
    """
    # -------------------------
    # Step 2: Estimate bin edges from centers
    # -------------------------
    n = len(x)
    edges = np.zeros(n + 1)

    # Left edge of the first bin
    edges[0] = x[0] - 0.5 * (x[1] - x[0])
    # Interior edges
    for i in range(1, n):
        edges[i] = 0.5 * (x[i] + x[i - 1])
    # Right edge of the last bin
    edges[-1] = x[-1] + 0.5 * (x[-1] - x[-2])

    # -------------------------
    # Step 3: Integrate using sum of y_i * bin_width_i
    # -------------------------
    bin_widths = edges[1:] - edges[:-1]
    integral_value = np.sum(y_mod * bin_widths)
    """

    integral_value = np.sum(y_mod[1:] * np.diff(x)) #drop y's first element

    return integral_value

In [ ]:
# read the file, output the data structure
def read_spenvis_csv_to_df(filename):
    reading_spectra=False
    reading_objects=False
    energy = None
    int_spectra_data = []
    diff_spectra_data = []
    bl_data=[]
    
    data={}
    linenumber=0
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            line = line.replace("'","")
#            print(linenumber,len(line.split(',')))
            # Identify the location in the file
            if line.startswith("PRJ_DEF"): #starting the section with tag, length, value, unit
                reading_objects=True
            elif line.startswith("Flux"):
                reading_objects=False
                reading_spectra=True
                continue
            elif line.startswith("End of File"):
                reading_spectra=False
                reading_objects=False
                break #done with file, end the loop
            if reading_objects and line.startswith("Mission start"):
                reading_objects=False
                continue
                

            if reading_spectra:
                # Convert the row of spectra values to floats and append
                row_values = [float(val) for val in line.split(',')[0:1]]
                bl_data.append(row_values)
                row_values = [float(val) for val in line.split(',')[2:]]
                int_spectra_data.append(row_values)
                diff_spectra_data.append(-np.diff(row_values)/np.diff(data['ENERGY']["values"]))
            if reading_objects and len(line.split(','))>1 and len(line.split(','))==int(line.split(',')[1])+3:
                name=line.split(',')[0]
                array_length=int(line.split(',')[1])
                values=[float(v) for v in line.split(',')[2:-1]] #get the values
                units=line.split(',')[-1]
                data[name]={"values": values, "units": units,"length":array_length}
            linenumber+=1
    data["bl"]={"values": bl_data, "units":"Gaus, L"}
    data["int_spectra"]={"values": int_spectra_data, "units":"cm^2 s^-1"}
    data["diff_spectra"]={"values": diff_spectra_data, "units":"cm^2 s^-1 MeV^-1"}
    print("Read ",len(data["int_spectra"]["values"]), "lines of spectra.")
    return data

In [ ]:
def plot_spectra_vs_energy(data, start=0, stop=130, step=1):
    """
    Plots every 'step'-th row of the 'spectra' vs. 'energy'.
    """
    plt.figure(figsize=(16, 6))  # width=12 inches, height=8 inches
    number_to_plot=int((stop-start)/step)
    cmap = plt.cm.get_cmap('hsv', int(stop-start+1))
    tot_neutron_count=0
    total_number_of_spectra=len(data["int_spectra"]["values"])
    for i in range(start, stop, step): 
        j=i%total_number_of_spectra #loop over 0-255
        energy=data['ENERGY']['values'][1:]
        spectrum=data['diff_spectra']['values'][j]
        spallation_neutron_count=integrate_spectrum_byEq1(energy,spectrum, threshold=200)
        plot_label=str(j)+"min, "+"neutron source count = "+f"{spallation_neutron_count:.1e}"+" s$^{-1}$"
#        plot_label=str(j)+"min"
        plt.plot(energy,spectrum, label=plot_label,color=cmap(j-start))
        tot_neutron_count+=spallation_neutron_count
    
    plt.xlabel('E [MeV]',fontsize=20)
    plt.ylabel('protons counts [cm$^{-2}$ s$^{-1}$ MeV$^{-1}$]',fontsize=20)
    plt.title('SPENVIS, AP-8 proton model, Differential flux')
    plt.yscale('log')
    plt.grid(True)
    plt.legend(loc="lower left")
    plt.show()
    return tot_neutron_count/number_to_plot # the average rate, s^-1

In [ ]:
def plot_neutrons_vs_time(data):
    duration=60*24*data['ORB_DUR']['values'][0] #in minutes
    spectrum=data['diff_spectra']['values']

    count=[]
    fig, ax1 = plt.subplots(figsize=(8, 5))  # Width=8, Height=5 inches
    
    for i in range(0, len(spectrum), 1): 
        energy=data['ENERGY']['values'][1:]
        spectrum=data['diff_spectra']['values'][i]
        spallation_neutron_count=integrate_spectrum_byEq1(energy,spectrum, threshold=200)
        count.append(spallation_neutron_count)
    ax1.plot(count,label="spallation neutrons")
    ax1.set_xlabel('time [min]]',fontsize=20)
    ax1.set_ylabel('neutron count [s$^{-1}$]',fontsize=20)
#    plt.yscale('log')
    ax1.grid(True)
    ax1.legend(loc="upper left")
    
    ax2 = ax1.twinx()
    matrix=np.array(data['int_spectra']['values'])
    matrix = np.clip(matrix, 0, None) #set all negative values to None
    ax2.plot(matrix[:,8], 'r--', label='protons, E>=1 MeV') #proton flux with E>=1MeV
    ax2.set_ylabel('protons, E>=1 MeV', color='r')
    ax2.tick_params(axis='y', labelcolor='r')
    ax2.legend(loc="upper right")
    
    plt.title('SPENVIS, AP-8 proton model')
    plt.show()
    return count

In [ ]:
if __name__ == "__main__":
    # Example usage:
    data = read_spenvis_csv_to_df('data.csv')
    rate=plot_spectra_vs_energy(data,308,324,1)
    plot_neutrons_vs_time(data)

In [ ]:
import math
emission_rate=rate
detection_rate=1
a=1# area of the detector in m^2, assuming 100% efficiency
distance = math.sqrt(emission_rate*a/(4*math.pi*detection_rate))
print(distance) #in meters

In [ ]:
matrix=np.array(data['int_spectra']['values'])
len(matrix[:,0])

In [ ]:
data['ENERGY']['values']

In [ ]:
#save the spectrum of a particular minute to a tsv file, for grasshopper
minute=316 # minute
output = np.column_stack((data['ENERGY']['values'][1:], data['diff_spectra']['values'][minute])) #total integral
output_string=f"{data['int_spectra']['values'][minute][1]:.0f}"
minute_string=str(minute)
filename="output_"+str(minute)+"_min_"+output_string+"_per_cm2_per_s.tsv"
np.savetxt(filename, output, delimiter="\t", fmt='%.2f')
print("Wrote to ",filename)